# Asynchronous and concurrent execution on GPUs
>*Melina Abeling, Julian Aeissen, Michele Pagani*. Supervised by *Oliver Fuhrer*

*GPUs allow for asynchronous (CPU simply launches work on GPU and then continues) and concurrent (multiple tasks are executed in parallel on the GPU) execution. In this project you will implement a simple stencil program and investigate performance using different stencil motifs as a function of grid size and amount of concurrency. It is foreseen to use either CuPy or CUDA for this project.*


---

## Introduction
*TODO*

---

## Methods
*TODO*

In [ ]:
# Imports
import time
import numpy as np
import cupy as cp
import matplotlib.pyplot as plt

In [ ]:
# Utils


### Stencils
*TODO*

#### A
*TODO*

In [ ]:
def step_stencil_a(field):
    pass

#### B
*TODO*

In [ ]:
def step_stencil_b(field):
    pass

### Sequential
*TODO*

In [ ]:
def sequential_computation(stencil, field):
    pass

### Concurrent
*TODO*

In [ ]:
def compute_gpu(stencil, input_field, n_stream):
    pass

### Initial field
*TODO*

In [ ]:
def get_initial_field(size):
    pass

---

## Results
*TODO*

### Performance over concurrency
*TODO*

In [ ]:
# Settings
steps_concurrency = [1, 2, 4, 8, 16, 32]
field_size = 512*512

# Setup
input_field_concur = get_initial_field(field_size)
output_fields_a_concur = []
output_fields_b_concur = []
times_a_concur = []
times_b_concur = []

for concurrency in steps_concurrency:

    # Compute for stencil A
    tic = time.perf_counter()
    output_fields_a_concur.append(compute_gpu(step_stencil_a, input_field_concur, concurrency))
    times_a_concur.append(time.perf_counter())

    # Compute for stencil B
    tic = time.perf_counter()
    output_fields_b_concur.append(compute_gpu(step_stencil_b, input_field_concur, concurrency))
    times_b_concur.append(time.perf_counter())


In [ ]:
# Plots
plt.title("Concurrency analysis")

plt.plot(steps_concurrency, times_a_concur, label = "Stencil A")
plt.plot(steps_concurrency, times_b_concur, label = "Stencil B")

plt.xlabel('# streams')
plt.ylabel('Time [s]')
plt.legend()


### Performance over grid size
*TODO*

In [ ]:
# TODO

In [ ]:
# Plots

---

## Discussion
*TODO*

---

## Conclusion
*TODO*